<a href="https://colab.research.google.com/github/carlosjviola/MeLi-challenge/blob/main/MeLi_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MeLi** - Data Visualization & Analytics Challenge

Carlos José Viola - 2022/11 

[Ver especificaciones](https://drive.google.com/file/d/1CTPIC5OIZ58joVDMG9AYAEecKTCsRtlc/view?usp=share_link)

Supuestos / Consideraciones

1) El archivo tiene registros correspondientes al mes de enero de 2022 para el campo first_login_dt

2) No hay especificaciones para el campo week_year. Se considera corresponde a la semana del año del campo first_login_dt

3) Se realizan distintas operaciones de limpieza de datos

3) Los registros del 1 y 2 de enero vienen con la semana 52. De acuerdo a lo supuesto en el punto anterior se reemplazarán los valores.

4) Removemos 

In [25]:

import pandas as pd
import numpy as np
from datetime import date

# Se crea una variable para almacenar nombre del archivo a analizar
archivoCsv = 'https://raw.githubusercontent.com/carlosjviola/MeLi-challenge/main/dataset_challenge_analytics.csv'
# Cargamos el archivo en un DataFrame de Pandas realizar el estudio
dfUsrOrig = pd.read_csv(archivoCsv)

#Vemos el tamaño del archivo (filas, columnas) y los primeros 5 registros del mismo
print("Cantidad de filas y columnas")
print(dfUsrOrig.shape)

print("\nMuestra las primeras 10 filas")
print(dfUsrOrig.head(10))

print("\nMuestra las últimas 10 filas")
print(dfUsrOrig.head(10))

Cantidad de filas y columnas
(11500, 12)

Muestra las primeras 10 filas
   Unnamed: 0 first_login_dt  week_year         user_id  habito habito_dt  \
0           0     2022-01-30          4   MLB2883855850     0.0       NaN   
1           1     2022-01-30          4   MLB7059024930     0.0       NaN   
2           2     2022-01-07          1   MLB4163269880     0.0       NaN   
3           3     2022-01-07          1   MLB7349806870     0.0       NaN   
4           4     2022-01-07          1   MLB7239064190     0.0       NaN   
5           5     2022-01-01         52  MLB10318536730     0.0       NaN   
6           6     2022-01-01         52   MLB7546352300     0.0       NaN   
7           7     2022-01-16          2   MLB2785023500     0.0       NaN   
8           8     2022-01-16          2   MLB8395954300     0.0       NaN   
9           9     2022-01-16          2   MLB1610695500     0.0       NaN   

   activacion activacion_dt  setup    setup_dt  return   return_dt  
0          

In [19]:
#Removemos las primera columna que contiene un id que entiendo no vamos a utilizar
dfUsr = dfUsrOrig.drop(columns = dfUsrOrig.columns[0], axis = 1, inplace= False)

print("El nuevo DataFrame tiene una columna menos")
print(dfUsr.shape)


El nuevo DataFrame tiene una columna menos
(11500, 11)


In [26]:
#Vamos a analizar la columna user_id y ordenamos esa columna
dfUsr.sort_values('user_id', inplace= True)
print(dfUsr['user_id'].describe())
print("\nObservamos una muestra aleatoris de 50 registros")
pd.set_option('display.max_rows', None)
print(dfUsr['user_id'].sample(n=50))

count               8500
unique              8500
top       MLB10001480800
freq                   1
Name: user_id, dtype: object

Observamos una muestra aleatoris de 100 registros
1772     MLB6049026390
9110     MLB2423233020
4441    MLB10630717810
5772       MLB13726250
5711     MLB8148067600
9696     MLB6855735120
8460    MLB10640308280
4974    MLB10644431440
1592    MLB10615189240
2351    MLB10611123330
4713     MLB3127065370
8123     MLB2816368520
1968    MLB10580813520
1686     MLB2150353280
1863    MLB10612447700
3188    MLB10434576810
2936     MLB5666673940
3716     MLB2658433150
7684     MLB1885036280
3815    MLB10433511230
5637    MLB10523219320
1674    MLB10506320740
2252      MLB288293160
2819    MLB10551494650
6973     MLB3476465700
3504     MLB2032961580
3705    MLB10609459130
2386     MLB6937307550
7551     MLB3993737980
7515     MLB2259016860
6494    MLB10645457910
4187     MLB4849162140
4609    MLB10011098280
3679    MLB10529354590
2691    MLB10498824810
4535     MLB382

In [27]:
#En algunos registros aparece '-1' al final lo que probablemente se trata de un error
#Reemplazamos los caracteres '-1' 
dfUsr['user_id'] = dfUsr['user_id'].str.replace('-1','')

#Vamos a verificar que el campo user_id tenga valores únicos
print("Cantidad total registros")
print(len(dfUsr['user_id']))

print("\nCandidad registros únicos")
print( len(dfUsr['user_id'].unique()) )

#Cómo han aparecido duplicados vamos a eliminarlos.
#Para esto creamos una serie que identifique los duplicados
lSerie = dfUsr['user_id'].duplicated(keep = False) 
 
dfUsr = dfUsr[~lSerie] 

print("\nNueva cantidad total registros sin duplicados")
print(len(dfUsr['user_id']))



Cantidad total registros
8500

Candidad registros únicos
8500

Nueva cantidad total registros sin duplicados
8500


In [29]:
# Convertimos la columna hábito en entero de acuerdo a las especificaciones
# antes llenamos con 0 los valores nulos en esta columna
display(dfUsr.dtypes)

dfUsr['habito'] = dfUsr['habito'].fillna(0.0).astype(int)
  
print('\nVerificamos los cambios')
display(dfUsr.dtypes)

first_login_dt    object
week_year          int64
user_id           object
habito             int64
habito_dt         object
activacion         int64
activacion_dt     object
setup              int64
setup_dt          object
return             int64
return_dt         object
dtype: object


Verificamos los cambios


first_login_dt    object
week_year          int64
user_id           object
habito             int64
habito_dt         object
activacion         int64
activacion_dt     object
setup              int64
setup_dt          object
return             int64
return_dt         object
dtype: object

In [30]:
# Convertimos en date los datos date-time en las columnas fecha. 
# Haciendo una inspección rápida del csv se observa que no hay horas
dfUsr['first_login_dt'] = pd.to_datetime(dfUsr['first_login_dt']).dt.date
dfUsr['habito_dt'] = pd.to_datetime(dfUsr['habito_dt']).dt.date
dfUsr['activacion_dt'] = pd.to_datetime(dfUsr['activacion_dt']).dt.date
dfUsr['setup_dt'] = pd.to_datetime(dfUsr['setup_dt']).dt.date
dfUsr['return_dt'] = pd.to_datetime(dfUsr['return_dt']).dt.date


In [31]:
# Como no está muy claro su contenido vamos a analizar la columna week_year
dfUsr['week_year'].agg(['mean', 'max', 'min'])

mean     4.348471
max     52.000000
min      1.000000
Name: week_year, dtype: float64

In [ ]:
#Los datos son de enero, no puede haber registros con el valor 52 en semana
print(dfUsr[dfUsr['week_year']==52])

     first_login_dt  week_year         user_id  habito habito_dt  activacion  \
2770     2022-01-02         52  MLB10004252630       0       NaT           1   
9577     2022-01-01         52  MLB10019564580       0       NaT           0   
5717     2022-01-01         52  MLB10064714530       0       NaT           0   
3170     2022-01-01         52  MLB10074070400       0       NaT           0   
4404     2022-01-02         52  MLB10088145650       0       NaT           1   
...             ...        ...             ...     ...       ...         ...   
4280     2022-01-02         52   MLB8381923230       0       NaT           1   
7115     2022-01-01         52    MLB852124930       0       NaT           1   
5591     2022-01-02         52    MLB938961270       0       NaT           1   
4761     2022-01-01         52    MLB949054360       0       NaT           1   
5733     2022-01-02         52    MLB950258540       0       NaT           0   

     activacion_dt  setup    setup_dt  

In [ ]:
#Vemos que los registros con semana 52 corresponden al 1 y 2 de enero
#Como son los primeros días del año quedaron con la última semana del anterior. 
#Debemos reemplazar 52 por 1


In [32]:
#dfError = dfUsr[ dfUsr['activacion_dt'] < dfUsr['first_login_dt']]
#Vamos a verificar que ninguna fecha en las columnas del archivo sea anterior a la fecha de primer login
#Creamos una nueva columna max_dt en el dataset con la mayor fecha entre activación, setup, habito y return

def calc_new_col(row):
    columnas = ['activacion_dt', 'setup_dt', 'habito_dt', 'return_dt']
    max_dt = date.min 
    for col in columnas:
        if row[col] > max_dt:
            max_dt = row[col]
    return max_dt

dfUsr["max_dt"] = dfUsr.apply(calc_new_col, axis=1)
print(dfUsr.head()) 
print(dfUsr.shape) 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  if __name__ == '__main__':


     first_login_dt  week_year         user_id  habito   habito_dt  \
5561     2022-01-13          2  MLB10001480800       0         NaT   
9631     2022-01-06          1  MLB10001530370       0         NaT   
5035     2022-01-05          1  MLB10002487550       1  2022-01-18   
8479     2022-01-07          1  MLB10002855700       0         NaT   
3189     2022-01-15          2  MLB10003112290       0         NaT   

      activacion activacion_dt  setup    setup_dt  return   return_dt  \
5561           1    2022-01-13      0         NaT       0         NaT   
9631           1    2022-01-06      0         NaT       1  2022-01-13   
5035           1    2022-01-07      1  2022-01-08       1  2022-01-06   
8479           0           NaT      0         NaT       0         NaT   
3189           0           NaT      0         NaT       0         NaT   

          max_dt  
5561  2022-01-13  
9631  2022-01-13  
5035  2022-01-18  
8479  0001-01-01  
3189  0001-01-01  
(8500, 12)


In [33]:
print('Tamaño del dataset')
print(dfUsr.shape) 

indexUsr = dfUsr[ dfUsr['max_dt'] < dfUsr['first_login_dt']].index
print('\nFilas con error de fecha')
print(indexUsr.shape) 

dfUsrFinal = dfUsr.drop(indexUsr)
print(dfUsrFinal.head(15))
print(dfUsrFinal.shape)


Tamaño del dataset
(8500, 12)

Filas con error de fecha
(1769,)
     first_login_dt  week_year         user_id  habito   habito_dt  \
5561     2022-01-13          2  MLB10001480800       0         NaT   
9631     2022-01-06          1  MLB10001530370       0         NaT   
5035     2022-01-05          1  MLB10002487550       1  2022-01-18   
3103     2022-01-19          3  MLB10003554750       1  2022-02-08   
7609     2022-01-21          3  MLB10003698160       0         NaT   
2770     2022-01-02         52  MLB10004252630       0         NaT   
4929     2022-01-19          3  MLB10004759110       0         NaT   
2343     2022-01-26          4  MLB10005576530       0         NaT   
5950     2022-01-10          2  MLB10006217790       1  2022-01-23   
7859     2022-01-20          3  MLB10007151650       0         NaT   
4609     2022-01-17          3  MLB10011098280       0         NaT   
4632     2022-01-07          1  MLB10012550470       0         NaT   
7167     2022-01-11       

In [38]:
dfUsrFinal['activacion_dias'] = (dfUsrFinal['activacion_dt']-dfUsrFinal['first_login_dt']).dt.days
dfUsrFinal['setup_dias'] = (dfUsrFinal['setup_dt']-dfUsrFinal['first_login_dt']).dt.days
dfUsrFinal['return_dias'] = (dfUsrFinal['return_dt']-dfUsrFinal['first_login_dt']).dt.days
print(dfUsrFinal.head(15))


     first_login_dt  week_year         user_id  habito   habito_dt  \
5561     2022-01-13          2  MLB10001480800       0         NaT   
9631     2022-01-06          1  MLB10001530370       0         NaT   
5035     2022-01-05          1  MLB10002487550       1  2022-01-18   
3103     2022-01-19          3  MLB10003554750       1  2022-02-08   
7609     2022-01-21          3  MLB10003698160       0         NaT   
2770     2022-01-02         52  MLB10004252630       0         NaT   
4929     2022-01-19          3  MLB10004759110       0         NaT   
2343     2022-01-26          4  MLB10005576530       0         NaT   
5950     2022-01-10          2  MLB10006217790       1  2022-01-23   
7859     2022-01-20          3  MLB10007151650       0         NaT   
4609     2022-01-17          3  MLB10011098280       0         NaT   
4632     2022-01-07          1  MLB10012550470       0         NaT   
7167     2022-01-11          2  MLB10014188850       1  2022-01-16   
7504     2022-01-07 

In [39]:
dfUsrFinal.to_csv('resultado2.csv')


# Nueva sección